In [1]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import kendalltau, spearmanr
from datetime import timedelta, datetime
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin
from pathlib import Path
from tqdm.notebook import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# data_path = Path('/content/drive/MyDrive/Colab_Notebooks/ad_ml')
data_path = Path('.')

tournaments_pkl = data_path / "tournaments.pkl"
players_pkl = data_path / "players.pkl"
results_pkl = data_path / "results.pkl"

# Data

In [3]:
def unpickle_(file_path: Path):
    with open(file_path, "rb") as pfile:
        unp_file = pickle.load(pfile)
    return unp_file

def to_pandas(file_path: Path):
    unp_file = unpickle_(file_path)
    df = pd.DataFrame.from_dict(unp_file.values())
    return df

players = to_pandas(players_pkl)
tourn = to_pandas(tournaments_pkl)
results = unpickle_(results_pkl)

In [4]:
tourn.dateStart = pd.to_datetime(tourn.dateStart, utc=True)
tourn.dateEnd = pd.to_datetime(tourn.dateEnd, utc=True)
tourn['qnum'] = tourn.loc[:, "questionQty"].apply(lambda x: x if x is None else sum(x.values()))


In [5]:
players.head()

,id,name,patronymic,surname
0,1,Алексей,None,Абабилов
1,10,Игорь,,Абалов
2,11,Наталья,Юрьевна,Абалымова
3,12,Артур,Евгеньевич,Абальян
4,13,Эрик,Евгеньевич,Абальян


Сразу проверим. Если данные правильные, то в них должен быть я.

In [6]:
players[players.surname.str.contains("Лисин") & players.name.str.contains("Фёдор")]

,id,name,patronymic,surname
80647,91415,Фёдор,Альбертович,Лисин


In [5]:
tourn_2019 = tourn[tourn.dateStart.dt.year.isin([2019])]
tourn_2020 = tourn[tourn.dateStart.dt.year.isin([2020])]

prop_tour_id_2019 = set(tourn_2019.id)
prop_tour_id_2020 = set(tourn_2020.id)

Есть проблема в данных: маски кое-где разной длины. Более того, маски могут оказаться больше по длине, чем определено в информации о турнирах. Так что делая one-hot-encoding просто будем пропускать подобные команды.

In [6]:
from typing import Tuple, List, TypeVar, Set
from collections import defaultdict
from tqdm.notebook import tqdm


T = TypeVar("T")

def clean_results(results: T, tids: Set[int], tourn: pd.DataFrame) -> Tuple[T, List[int], List[int], List[int]]:
    """
    Return changed results, list of available tournaments, list of players
    and list of commands (ids).
    """
    teams: Set[int] = set()
    players: Set[int] = set()
    tournaments: Set[int] = set()
    cleaned_results = defaultdict(list)

    for tid in tqdm(results.keys(), desc='Process games', total=len(results)):
        if tid not in tids:
            continue
        for team in results[tid]:
            try:
                if (team['mask'] is not None) and len(team['teamMembers']) > 0:
                    est_mask_len = tourn[tourn.id == tid]['qnum'].item()
                    if len(team['mask']) != est_mask_len:
                        continue
                    cleaned_results[tid].append(team.copy())
                    tournaments.add(tid)
                    teams.add(team['team']['id'])
                    
                    for member in team['teamMembers']:
                        players.add(member['player']['id'])
            except KeyError:
                continue
    
    return dict(cleaned_results), list(tournaments), list(players), list(teams)

# Baseline

Обучим линейную регрессию предсказывать возьмет ли команда вопрос или нет. В качестве фичей выберем вектор, в котором будет храниться информация о составе команды и решаемом вопросе. В итоге получится сильно разряженная матрица из 0 и 1.

В обученной регрессии веса будут отвечать за сложность вопроса и силу игрока. В качестве ранжирования команд по силе для 2020 года будем использовать сумму соответсвующих весов.

In [21]:
results_2019, tournaments_ids, players_id, teams_id = clean_results(results, set(tourn_2019.id), tourn_2019)

Process games::   0%|          | 0/5528 [00:00<?, ?it/s]

In [22]:
player_to_id = {old_id: new_id for new_id, old_id in enumerate(players_id)}
id_to_player = {nid: oid for oid, nid in player_to_id.items()}

assert len(player_to_id) == len(players_id)

In [23]:
tourn_2019.set_index("id", inplace=True)

## Данные

In [26]:
question_to_id = {}
# id_to_question = {}
global_count = len(players_id)

for tid, tour in tourn_2019.iterrows():
    for qi in range(int(tour.qnum)):
        name = f"{tid}_{qi}"
        question_to_id[name] = global_count
        # id_to_question[global_count] = name
        global_count += 1

assert global_count == len(question_to_id) + len(players_id)

In [27]:
from scipy.sparse import lil_matrix

# create dataset
total = 0

for tid in tournaments_ids:
    for team in results_2019[tid]:
        total += len(team['mask'])

print(total)

targets = []
data = lil_matrix((total, global_count), dtype=np.uint8)

3340549


In [28]:
def get_team_members(team, p2id):
    res = []
    for tm in team['teamMembers']:
        res.append(p2id[tm['player']['id']])
    return sorted(res)

In [30]:
global_step = 0

for tid in tqdm(tournaments_ids):
    for team in results_2019[tid]:
        for qnum, target in enumerate(team['mask']):
            if not target.isnumeric():
                global_step += 1
                targets.append(0)
                continue
            if qnum >= tourn_2019.loc[tid, "qnum"]:
                global_step += 1
                targets.append(0)
                continue
            q_name = f"{tid}_{qnum}"
            qid = question_to_id[q_name]
            targets.append(int(target))
            members = get_team_members(team, player_to_id)
            for mid in members:
                data[global_step, mid] = 1
            data[global_step, qid] = 1
            global_step += 1

  0%|          | 0/677 [00:00<?, ?it/s]

In [31]:
data = data.tocsr()
len(targets)

3340549

Линейная регрессия выбрана, потому что логика получения предсказания таргета - скалярное произведение. Тогда вес игрока - его аддитивный вклад в получение верного ответа.

In [32]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(data, targets)

LinearRegression()

In [33]:
player_w = model.coef_[:len(players_id)]

## Метрики

In [37]:
results_2020, tourn2020_ids, _, _ = clean_results(results, set(tourn_2020.id), tourn_2020)

Process games::   0%|          | 0/5528 [00:00<?, ?it/s]

In [7]:
def get_ratings(tid, results, players_W, player_to_id):
    res = []
    target = []
    orig_order = []

    for team in results[tid]:
        score = 0
        for mem in team['teamMembers']:
            mid = player_to_id.get(mem['player']['id'], None)
            score += players_W[mid] if mid is not None else 0

        res.append((team['team']['id'], score))
        target.append(team['position'])
        orig_order.append(team['team']['id'])

    res = sorted(res, key=lambda x: -x[1])
    res_d = {team_id : i + 1 for i, (team_id, _) in enumerate(res)}
    pred = [res_d[t] for t in orig_order]
    return pred, target

def mean_correlations(tournaments_ids, results, players_W, player_to_id):
    score_spearmanr = []
    score_kendalltau = []
    spr_nans = 0
    kdl_nans = 0

    for tid in tqdm(tournaments_ids):
        pred, target = get_ratings(tid, results, players_W, player_to_id)
        spr = spearmanr(pred, target)[0]
        if spr is np.nan:
            spr_nans += 1
        else:
            score_spearmanr.append(spr)

        kdl = kendalltau(pred, target)[0]
        if spr is np.nan:
            kdl_nans += 1
        else:
            score_kendalltau.append(kdl)

    print(f"{spr_nans} spearmanr scores is none")
    print(f"{kdl_nans} kendalltau scores is none")

    print(f"Mean spearmanr: {np.array(score_spearmanr).mean()}")
    print(f"Mean kendalltau: {np.array(score_kendalltau).mean()}")

    return score_spearmanr, score_kendalltau

In [39]:
spr, kdl = mean_correlations(tourn2020_ids, results_2020, player_w, player_to_id)

  0%|          | 0/170 [00:00<?, ?it/s]

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.7141019139415572
Mean kendalltau: 0.5608064504868062


# EM

## Not working solution

Сначала попробуем реализовать EM алгоритм на уже сформированных данных. Т.е. предсказания будем делать для строки фичей для всей команды на конкретном вопросе.

Допущения, которые будут сохраняться для всех EM алгоримов ниже:
- Если в команде кто-то знает ответ, то вся команда отвечает правильно
- Если команда ответила неверно, то любой в команде не знает ответа
- Игроки независимо знают/не знают ответы

В общем утверждения неверны. Судя по моим эмпирическим данным, в камандах чаще всего есть капитан и нет демократии. Так что в спорных или сложных вопросах именно капитан решает что сдавать в качестве ответа. А значит существет bias как минимум в те моменты, когда капитан уверен в своём ответе, а ответов у команды несколько.

In [40]:
num_epochs = 5
y_pred = np.array(targets)
mask = (y_pred == 0)

for ep in tqdm(range(num_epochs), total=num_epochs):
    model = LinearRegression()
    model.fit(data, y_pred)
    player_w = model.coef_[:len(players_id)]

    print(f"Epoch #{ep}")
    mean_correlations(tourn2020_ids, results_2020, player_w, player_to_id)

    y_pred = model.predict(data)
    y_pred[mask] = 0
    # y_pred[y_pred > 1] = 1 # можно sigmoid, можно подвигать threshold


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #0


  0%|          | 0/170 [00:00<?, ?it/s]

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.7141019139415572
Mean kendalltau: 0.5608064504868062
Epoch #1


  0%|          | 0/170 [00:00<?, ?it/s]

...

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.6899124728293284
Mean kendalltau: 0.5352270328150917
Epoch #3


  0%|          | 0/170 [00:00<?, ?it/s]

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.6734019692961727
Mean kendalltau: 0.5206238340515554
Epoch #4


  0%|          | 0/170 [00:00<?, ?it/s]

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.6603146240403236
Mean kendalltau: 0.5094071227356204


Идея модели выше - предсказывать вероятность в текущем составе правильно ответить на вопрос. Качество падает при обучении EM алгоритмом. Полагаю это из-за того, что модель предсказывает для всей команды сразу, а число матчей игрока в разных командах недостаточно, чтобы модель правильно подобрала его "силу". Т.е. команды (вероятнее) модель ранжировала бы лучше, чем ранжирует людей. 

Ну или потому что модель обучает неведомо на чем, потому что теоретические выводы я не делал. Просто поигрался.
Но проверять я этого пока не хочу, и так все дедлайны уже пропустил.

Попробуем построить модель, которая предсказывает вероятность игрока ответить на вопрос. Скрытыми параметрами будут ответы других игроков. На первом шаге считаем, что если команда ответила верно, то искомая вероятность 1. Тогда моделировать мы хотим вероятность того, что игрок дал верный ответ, если команда дала верный ответ: $P(x_{ij}=1|t_i=1)$

Из допущений и описания получаем следующее:
- предсказание модели на M-шаге это вероятность игрока ответить правильно $P(x_{ij}=1)$ (простая сигмоида от выхода линейной модели)
- $P(t_i=1|x_{ij}=1)=1$ - вероятность команды дать ответ на $i$, если ответ знает игрок $j$;
- вероятность команды дать ответ $P(t_i=1) = 1 - \prod P(z_{ij} \neq 1)$


$$P(x_{ij}=1|t_i=1) = \frac{P(x_{ij}=1)}{P(t_i=1)}$$

## Data

Пересоберём данные.

In [8]:
results_2019, tourn_ids_2019, players_2019, commands_2019 = clean_results(results, prop_tour_id_2019, tourn_2019)

Process games:   0%|          | 0/5528 [00:00<?, ?it/s]

In [9]:
results_2020, tourn_ids_2020, players_2020, commands_2020 = clean_results(results, prop_tour_id_2020, tourn_2020)

Process games:   0%|          | 0/5528 [00:00<?, ?it/s]

In [10]:
df = tourn_2019[tourn_2019['id'].isin(tourn_ids_2019)].loc[:, ["id", "qnum"]]
# df.set_index("id", inplace=True)

In [11]:
def add_all_info(df: pd.DataFrame, cleaned_results):    
    teams_ids_per_row = []
    members_per_team_per_row = []
    targets_per_member_per_team_per_row = []
    questions_per_member_per_team_per_row = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        tid = row.id
        tour_teams_ids = []
        tour_members = []
        tour_questions = []
        tour_targets = []


        for team in cleaned_results[tid]:
            team_members = []
            team_questions = []
            team_targets = []
            team_id = team['team']['id']
            mask = team['mask']

            for member in team['teamMembers']:
                member_targets = []
                member_questions = []
                mid = member['player']['id']

                for qid, target in enumerate(mask):
                    if target not in ['0', '1']:
                        target = '0'
                    member_questions.append(f"{tid}_{qid}")
                    member_targets.append(int(target))
                
                team_members.append(mid)
                team_targets.append(member_targets)
                team_questions.append(member_questions)
            
            tour_teams_ids.append(team_id)
            tour_members.append(team_members)
            tour_questions.append(team_questions)
            tour_targets.append(team_targets)
        
        teams_ids_per_row.append(tour_teams_ids)
        members_per_team_per_row.append(tour_members)
        targets_per_member_per_team_per_row.append(tour_targets)
        questions_per_member_per_team_per_row.append(tour_questions)
    
    assert len(teams_ids_per_row) == len(members_per_team_per_row) == len(targets_per_member_per_team_per_row) == len(questions_per_member_per_team_per_row)

    df["team_id"] = teams_ids_per_row
    df["player_id"] = members_per_team_per_row
    df["question_id"] = questions_per_member_per_team_per_row
    df["target"] = targets_per_member_per_team_per_row

    df = df.explode(['team_id', 'player_id', 'question_id', 'target'])
    df = df.explode(['player_id', 'question_id', 'target'])
    df = df.explode(['question_id', 'target'])
    df.reset_index(drop=True, inplace=True)
    # df = df.sort_values(["player_id", "question_id"])

    return df


In [12]:
df = add_all_info(df, results_2019)
df.head()

  0%|          | 0/677 [00:00<?, ?it/s]

,id,qnum,team_id,player_id,question_id,target
0,4772,36.0,45556,6212,4772.0_0,1
1,4772,36.0,45556,6212,4772.0_1,1
2,4772,36.0,45556,6212,4772.0_2,1
3,4772,36.0,45556,6212,4772.0_3,1
4,4772,36.0,45556,6212,4772.0_4,1


## Алгоритм

In [13]:
def sigmoid(x: np.ndarray):
  return 1 / (1 + np.exp(-x))

def inv_sigmoid(x: np.ndarray, eps: float = 1e-8):
  return np.log(x / (1 - x) + eps)


In [14]:
df['player_id'] = df.loc[:, 'player_id'].astype('category')
X_train = pd.get_dummies(
    df.loc[:, ['player_id', 'question_id']],
    sparse = True
)

In [15]:
to_int_id = lambda x: int(x.split('_')[-1])
player_to_id = {to_int_id(pid): pos for pos, pid in enumerate(X_train.columns) if "player" in pid}

In [16]:
num_epoch = 5
y_current = df.target.values.copy()
mask = (y_current == 0)
model = LinearRegression()


for epoch in tqdm(range(num_epoch), total=num_epoch):
    model.fit(X_train, y_current)
    players_w = model.coef_[:len(player_to_id)]
    mean_correlations(tourn_ids_2020, results_2020, players_w, player_to_id)

    logits = model.predict(X_train)
    probs = sigmoid(logits)
    np.putmask(probs, mask, 0)

    df_meta = df.loc[:, ["team_id", "question_id"]].copy()
    df_meta['inv_prob'] = 1 - probs

    df_team = df_meta.groupby(["team_id", "question_id"])['inv_prob'].prod().reset_index()
    df_team['prob_team'] = 1 - df_team['inv_prob']

    df_meta = df_meta.merge(
        df_team.loc[:, ["team_id", "question_id", "prob_team"]],
        on=['team_id', 'question_id']
    )

    y_current = (1 - df_meta['inv_prob'].values) / (df_meta['prob_team'].values + 1e-8)
    y_current = inv_sigmoid(y_current)

    

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/170 [00:00<?, ?it/s]

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.7679089181947956
Mean kendalltau: 0.6136665576492055


  0%|          | 0/170 [00:00<?, ?it/s]

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.771213405939426
Mean kendalltau: 0.6178774814127066


...

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.7728145074224528
Mean kendalltau: 0.6183204421871941


  0%|          | 0/170 [00:00<?, ?it/s]

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.7741083340184081
Mean kendalltau: 0.6192941939666139


  0%|          | 0/170 [00:00<?, ?it/s]

1 spearmanr scores is none
1 kendalltau scores is none
Mean spearmanr: 0.7741624938360204
Mean kendalltau: 0.6193387845235105


Ура победа!

# Поизучаем игроков, вопросы и турниры

In [19]:
from collections import defaultdict


idx_to_quest = {pos: name for pos, name in enumerate(X_train.columns) if "question" in name}
idx_to_player = {pos: pid for pid, pos in player_to_id.items()}
pl_dct = defaultdict(list)
q_dct = defaultdict(list)


for pos, pid in idx_to_player.items():
    pl_dct["player_id"].append(pid)
    pl_dct["power"].append(model.coef_[pos])

for pos, name in idx_to_quest.items():
    _, _, tid, qid = name.split("_")
    q_dct["question_id"].append(int(qid))
    q_dct["dif"].append(model.coef_[pos])
    q_dct["tournament_id"].append(int(float(tid)))


In [22]:
pl_w = pd.DataFrame.from_dict(pl_dct)
quest_w = pd.DataFrame.from_dict(q_dct)


## Турниры

In [80]:
df_tourn = tourn_2019[tourn_2019.id.isin(tourn_ids_2019)][["id", "name"]]
grouped_quest = quest_w.groupby(['tournament_id'])['dif'].mean().reset_index()

df_tourn = df_tourn.merge(
    grouped_quest, left_on=["id"], right_on=["tournament_id"],
)[["name", "dif", "tournament_id"]].sort_values("dif")

In [81]:
df_tourn.head(10)

,name,dif,tournament_id
665,Чемпионат Санкт-Петербурга. Первая лига,-9.277635,6149
543,Угрюмый Ёрш,-6.661209,5928
369,Синхрон высшей лиги Москвы,-5.877891,5684
641,Воображаемый музей,-5.600278,6101
43,Первенство правого полушария,-5.553577,5159
554,Чемпионат Мира. Этап 2. Группа В,-5.256732,5942
377,Знание – Сила VI,-5.208906,5693
179,Чемпионат России,-5.205867,5465
553,Чемпионат Мира. Этап 2. Группа А,-5.101974,5941
26,Ускользающая сова,-5.088549,5083


In [39]:
df_tourn.sort_values("dif", ascending=False).head(10)

,name,dif
642,One ring - async,11.378573
11,(а)Синхрон-lite. Лига старта. Эпизод V,7.649055
548,Школьная лига. I тур.,7.329599
566,Школьная лига. III тур.,7.313798
675,Школьная лига,7.200633
172,Студенческий чемпионат Калининградской области,7.126548
10,Школьный Синхрон-lite. Выпуск 2.5,7.116851
154,Синхрон Лиги Разума,6.983400
380,(а)Синхрон-lite. Лига старта. Эпизод VII,6.927060
565,Школьная лига. II тур.,6.919022


Веса получились тем меньше, чем сложнее вопросы, т.к. в обучение модели сложность вопроса делается аддитивный вклад в финальное предсказание логитов.

В первой десятке лежат Чемпионаты мира, Чемпионат Санкт-Петербурга. В последней десятке много школьных туров. Так что в целом выглядит правдоподобно.

## Игроки

In [43]:
df_players = players[players.id.isin(players_2019)]

In [48]:
pl_rating = defaultdict(list)
pl_ans = defaultdict(int)

for tid in tourn_ids_2019:
    for team in results_2019[tid]:
        mask = team["mask"]
        count = sum([int(i) for i in mask if i == '1'])
        for member in team['teamMembers']:
            pid = member['player']['id']
            rating = member['rating']
            pl_rating['player_id'].append(pid)
            pl_rating['rating'].append(rating)
            pl_ans[pid] += count


In [56]:
rating = pd.DataFrame.from_dict(pl_rating)
ans_count = pd.DataFrame.from_dict(pl_ans, orient='index', columns=["ans_count"]).reset_index(names=['player_id'])

In [63]:
df_players = df_players.merge(
    pl_w, left_on="id", right_on="player_id"
).merge(
    rating, on="player_id",
).merge(
    ans_count, on="player_id"
).drop(["id", "patronymic"], axis=1)

In [75]:
df_players.sort_values("power", ascending=False).head(20)

,name,surname,player_id,power,rating,ans_count
91649,Максим,Пилипенко,38175,19.659356,1670,29
53003,Илья,Немец,22474,18.231241,6350,45
53004,Илья,Немец,22474,18.231241,6775,45
379870,Елена,Бровченко,199963,15.881269,0,32
408709,Майя,Губина,212663,15.381457,0,50
408708,Майя,Губина,212663,15.381457,0,50
34355,Ольга,Козлова,14996,15.124418,887,27
95484,Дмитрий,Кудинов,40411,14.648554,7045,33
217234,Сергей,Завьялов,123750,14.439004,20,28
11498,Владимир,Вакуленко,4876,13.928242,0,28


In [74]:
df_players.sort_values("power", ascending=True).head(10)

,name,surname,player_id,power,rating,ans_count
401867,Карина,Сезько,209106,-10.937547,0,0
400377,Кристина,Ражко,208538,-10.937547,0,0
400174,Маргарита,Лущикова,208439,-10.937547,0,0
396333,Вероника,Манузина,207016,-10.937547,0,0
400175,Виталий,Григорьев,208440,-10.937547,0,0
400176,Семён,Счётников,208441,-10.937547,0,0
400177,Иван,Корнилов,208442,-10.937547,0,0
400173,Ксения,Косинова,208438,-10.937547,0,0
396846,Арсений,Урезалов,207183,-10.937547,0,0
396843,Анастасия,Якина,207180,-10.937547,0,0


## Вопросы

Тут я сначала думал, что все будет просто: запросить по API данные и посмотреть. Но оказывается база вопросов и база рейтинга не синхронизированны. А значит id совсем не совпадают. Значит надо парсить и искать по ключевым словам, что мне делать лень. Так что вот 10 самых сложных вопросов по версии моей рейтинговой системы за 2019 год.

Более того ЧМ в Баку имеет 4 этапа по два тура, а на сайте указано просто 8 туров. Сопоставить было не сложно, но понять...

1. Баку, Чемпионат мира, Финал, Группа А. Вопрос №14
    
    ![image.png](https://i.imgur.com/R1AzXEb.png)

    ```Слова ПЕРВЫЕ и ВТОРОЙ – замены. В шестидесятые годы ПЕРВЫМ давали ВТОРОЙ, надеясь «взломать защиту» и «освободить узников». Какой актёр изображён на раздаточном материале?```


2. Баку, Чемпионат Мира. Этап 2. Группа А. Вопрос №13.

    ```Персонажи сериала «Удивительная миссис Мэйзел» — интеллигентные люди, и к тому же вовсе не немцы. Возможно, именно поэтому, чокаясь в парижском кафе, они в шутку произносят односложное слово. Напишите это слово.```


3. Баку, Чемпионат Мира. Этап 3. Группа А. Вопрос №14.

    ```В начале XX века ресторатор Альфонс Котилс избежал заключения, пообещав, что будет вести бизнес только по телефону. Анна Сейфе называет его «ТАКИМ Альфонсом». Каким ТАКИМ?```
    

4. Баку, Чемпионат Мира. Этап 1. Группа А. Вопрос №4.

    ```Один купец осуществлял поставки тайно, так как производители косметики не хотели, чтобы покупатели узнали об использовании более дешёвого сырья вместо дорогой рисовой муки. Эта скрытность на некоторое время отсрочила появление европейского... ЧЕГО?```


5. Баку, Чемпионат Мира. Этап 2. Группа А. Вопрос №6.

    ```Прослушайте перевод первых строк одной поэмы: «Ах! Словно золото, падают листья на ветру; длинные годы бесчисленны, как крылья деревьев!» Считается, что в языке, на котором она была написана, 10 падежей, в том числе так называемый «инфинитив». Какое слово мы изменили в этом вопросе?```


6. Баку, Чемпионат Мира. Этап 3. Группа А. Вопрос №23.

    ```Инициированная экологами в начале 1990-х кампания по защите привела к тому, что в течение десяти лет численность сайгаков сократилась с миллиона до тридцати тысяч особей, преимущественно самок. Какое двукоренное слово мы пропустили в предыдущем предложении?```


7. Москва, Чемпионат России. Вопрос №39.
    
    ```По мнению Закари Смита, необычная шахматная партия в произведении 60-х годов двадцатого века не только должна была показать интеллектуальное превосходство, но и могла быть предвестником последующих событий. Назовите победителя этой партии, использовав аббревиатуру.```


8. Москва, Чемпионат России. Вопрос №67.

    ```Присутствие ЕЁ положительно сказывается также, например, на слонятах. На изготовление чьей ЕЁ пошёл кусок ткани с ромашками?```

9. Баку, Чемпионат Мира. Этап 1. Группа А. Вопрос №27.

    ```В послевоенном СССР зажигалки были редкостью, люди обходились спичками. В 1958 году ОН призвал развернуть массовое производство зажигалок – от дешёвых до дорогих подарочных. Если верить ЕМУ же, появившаяся вскоре модель зажигалок получила ЕГО имя. В одном из слов мы пропустили 4 буквы. Напишите это слово полностью.```

10. Баку, Чемпионат мира, Финал, Группа А. Вопрос №23.

    ```Айзек Азимов выражал опасения, упомянув при этом мотосани. Назовите человека, отклонившего в 1964 году такие опасения, упомянув при этом пемзу.```

## Ответы

А ответы немного замариновались. Так что стоит потрудиться, чтобы разобраться что к чему. Ну или позовите Рика Санчеза.

\x80\x04\x95t\x1b\x00\x00\x00\x00\x00\x00Xm\x1b\x00\x00\n1. \xd0\x94\xd0\xb0\xd1\x81\xd1\x82\xd0\xb8\xd0\xbd \xd0\xa5\xd0\xbe\xd1\x84\xd1\x84\xd0\xbc\xd0\xb0\xd0\xbd\n\n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\x9f\xd0\x95\xd0\xa0\xd0\x92\xd0\xab\xd0\x95 \xe2\x80\x93 \xd0\xb0\xd1\x83\xd1\x82\xd0\xb8\xd1\x81\xd1\x82\xd1\x8b, \xd0\x92\xd0\xa2\xd0\x9e\xd0\xa0\xd0\x9e\xd0\x99 \xe2\x80\x93 \xd0\x9b\xd0\xa1\xd0\x94. \xd0\x92 \xd1\x88\xd0\xb5\xd1\x81\xd1\x82\xd0\xb8\xd0\xb4\xd0\xb5\xd1\x81\xd1\x8f\xd1\x82\xd1\x8b\xd0\xb5 \xd0\xb3\xd0\xbe\xd0\xb4\xd1\x8b \xd0\xb4\xd0\xb5\xd1\x82\xd1\x8f\xd0\xbc \xd1\x81 \xd0\xb0\xd1\x83\xd1\x82\xd0\xb8\xd0\xb7\xd0\xbc\xd0\xbe\xd0\xbc \xd0\xb4\xd0\xb0\xd0\xb2\xd0\xb0\xd0\xbb\xd0\xb8 \xd0\x9b\xd0\xa1\xd0\x94, \xd1\x87\xd1\x82\xd0\xbe\xd0\xb1\xd1\x8b \xd0\xb2\xd0\xb7\xd0\xbb\xd0\xbe\xd0\xbc\xd0\xb0\xd1\x82\xd1\x8c \xc2\xab\xd0\xb0\xd1\x83\xd1\x82\xd0\xb8\xd1\x81\xd1\x82\xd0\xb8\xd1\x87\xd0\xb5\xd1\x81\xd0\xba\xd1\x83\xd1\x8e \xd0\xb7\xd0\xb0\xd1\x89\xd0\xb8\xd1\x82\xd1\x83\xc2\xbb \xd0\xb8 \xc2\xab\xd0\xbe\xd1\x81\xd0\xb2\xd0\xbe\xd0\xb1\xd0\xbe\xd0\xb4\xd0\xb8\xd1\x82\xd1\x8c \xd1\x80\xd0\xb0\xd0\xb7\xd1\x83\xd0\xbc \xd1\x80\xd0\xb5\xd0\xb1\xd1\x91\xd0\xbd\xd0\xba\xd0\xb0\xc2\xbb. \xd0\xa8\xd0\xb2\xd0\xb5\xd0\xb9\xd1\x86\xd0\xb0\xd1\x80\xd1\x81\xd0\xba\xd0\xb8\xd0\xb9 \xd1\x85\xd0\xb8\xd0\xbc\xd0\xb8\xd0\xba \xd0\x90\xd0\xbb\xd1\x8c\xd0\xb1\xd0\xb5\xd1\x80\xd1\x82 \xd0\xa5\xd0\xbe\xd1\x84\xd0\xbc\xd0\xb0\xd0\xbd\xd0\xbd, \xd1\x81\xd1\x87\xd0\xb8\xd1\x82\xd0\xb0\xd1\x8e\xd1\x89\xd0\xb8\xd0\xb9\xd1\x81\xd1\x8f \xc2\xab\xd0\xbe\xd1\x82\xd1\x86\xd0\xbe\xd0\xbc\xc2\xbb \xd0\x9b\xd0\xa1\xd0\x94, \xd0\xb8\xd1\x81\xd0\xbf\xd1\x8b\xd1\x82\xd0\xb0\xd0\xbb \xd0\xbd\xd0\xb0 \xd1\x81\xd0\xb5\xd0\xb1\xd0\xb5 \xd0\xb5\xd0\xb3\xd0\xbe \xd0\xb4\xd0\xb5\xd0\xb9\xd1\x81\xd1\x82\xd0\xb2\xd0\xb8\xd0\xb5 \xd0\xb2\xd0\xbe \xd0\xb2\xd1\x80\xd0\xb5\xd0\xbc\xd1\x8f \xd0\xbf\xd0\xbe\xd0\xb5\xd0\xb7\xd0\xb4\xd0\xba\xd0\xb8 \xd0\xbd\xd0\xb0 \xd0\xb2\xd0\xb5\xd0\xbb\xd0\xbe\xd1\x81\xd0\xb8\xd0\xbf\xd0\xb5\xd0\xb4\xd0\xb5. 19 \xd0\xb0\xd0\xbf\xd1\x80\xd0\xb5\xd0\xbb\xd1\x8f 1943 \xd0\xb3\xd0\xbe\xd0\xb4\xd0\xb0 \xd0\xb8\xd0\xb7\xd0\xb2\xd0\xb5\xd1\x81\xd1\x82\xd0\xb5\xd0\xbd \xd0\xba\xd0\xb0\xd0\xba \xd0\x94\xd0\xb5\xd0\xbd\xd1\x8c \xd0\xb2\xd0\xb5\xd0\xbb\xd0\xbe\xd1\x81\xd0\xb8\xd0\xbf\xd0\xb5\xd0\xb4\xd0\xb0. \xd0\x97\xd0\xb0 \xd1\x80\xd0\xbe\xd0\xbb\xd1\x8c \xd0\xb0\xd1\x83\xd1\x82\xd0\xb8\xd1\x81\xd1\x82\xd0\xb0 \xd0\xa0\xd1\x8d\xd0\xb9\xd0\xbc\xd0\xbe\xd0\xbd\xd0\xb4\xd0\xb0 \xd0\x91\xd1\x8d\xcc\x81\xd0\xb1\xd0\xb1\xd0\xb8\xd1\x82\xd0\xb0 \xd0\xb2 \xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c\xd0\xbc\xd0\xb5 \xc2\xab\xd0\xa7\xd0\xb5\xd0\xbb\xd0\xbe\xd0\xb2\xd0\xb5\xd0\xba \xd0\xb4\xd0\xbe\xd0\xb6\xd0\xb4\xd1\x8f\xc2\xbb \xd0\x94\xd0\xb0\xd1\x81\xd1\x82\xd0\xb8\xd0\xbd \xd0\xa5\xd0\xbe\xd1\x84\xd1\x84\xd0\xbc\xd0\xb0\xd0\xbd \xd0\xbf\xd0\xbe\xd0\xbb\xd1\x83\xd1\x87\xd0\xb8\xd0\xbb \xd0\xbf\xd1\x80\xd0\xb5\xd0\xbc\xd0\xb8\xd1\x8e \xc2\xab\xd0\x9e\xd1\x81\xd0\xba\xd0\xb0\xd1\x80\xc2\xbb.\n\n2. \xd0\x9f\xd1\x80\xd1\x83\xd1\x81\xd1\x82\n\n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\x9f\xd0\xbe\xd1\x81\xd0\xba\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd1\x83 \xd0\xb3\xd0\xb5\xd1\x80\xd0\xbe\xd0\xb8 \xd0\xb2\xd0\xbe\xd0\xb2\xd1\x81\xd0\xb5 \xd0\xb4\xd0\xb0\xd0\xb6\xd0\xb5 \xd0\xbd\xd0\xb5 \xd0\xbd\xd0\xb5\xd0\xbc\xd1\x86\xd1\x8b, \xd0\xb0 \xd0\xb8\xd0\xbd\xd1\x82\xd0\xb5\xd0\xbb\xd0\xbb\xd0\xb8\xd0\xb3\xd0\xb5\xd0\xbd\xd1\x82\xd0\xbd\xd1\x8b\xd0\xb5 \xd0\xb0\xd0\xbc\xd0\xb5\xd1\x80\xd0\xb8\xd0\xba\xd0\xb0\xd0\xbd\xd1\x86\xd1\x8b, \xd0\xb4\xd0\xb0 \xd0\xb5\xd1\x89\xd0\xb5 \xd0\xb8 \xd0\xbd\xd0\xb0\xd1\x85\xd0\xbe\xd0\xb4\xd1\x8f\xd1\x89\xd0\xb8\xd0\xb5\xd1\x81\xd1\x8f \xd0\xb2 \xd0\x9f\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb6\xd0\xb5, \xd0\xb2\xd0\xbc\xd0\xb5\xd1\x81\xd1\x82\xd0\xbe \xd1\x82\xd1\x80\xd0\xb0\xd0\xb4\xd0\xb8\xd1\x86\xd0\xb8\xd0\xbe\xd0\xbd\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\xbd\xd0\xb5\xd0\xbc\xd0\xb5\xd1\x86\xd0\xba\xd0\xbe\xd0\xb9 \xd0\xb7\xd0\xb4\xd1\x80\xd0\xb0\xd0\xb2\xd0\xb8\xd1\x86\xd1\x8b \xc2\xab\xd0\x9f\xd1\x80\xd0\xbe\xd1\x81\xd1\x82!\xc2\xbb \xd0\xbe\xd0\xbd\xd0\xb8 \xd0\xb2 \xd1\x88\xd1\x83\xd1\x82\xd0\xba\xd1\x83 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb8\xd0\xb7\xd0\xbd\xd0\xbe\xd1\x81\xd1\x8f\xd1\x82 \xc2\xab\xd0\x9f\xd1\x80\xd1\x83\xd1\x81\xd1\x82!\xc2\xbb.\n\n3. \xd1\x82\xd0\xb8\xd1\x84\xd0\xbe\xd0\xb7\xd0\xbd\xd1\x8b\xd0\xbc\n\n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\x92 \xd0\xbd\xd0\xb0\xd1\x87\xd0\xb0\xd0\xbb\xd0\xb5 XX \xd0\xb2\xd0\xb5\xd0\xba\xd0\xb0 \xd0\xba\xd1\x83\xd1\x85\xd0\xb0\xd1\x80\xd0\xba\xd0\xb0 \xd0\x9c\xd1\x8d\xd1\x80\xd0\xb8 \xd0\x9c\xd0\xb0\xd0\xbb\xd0\xbb\xd0\xbe\xd0\xbd, \xd0\xb1\xd0\xbe\xd0\xbb\xd0\xb5\xd0\xb5 \xd0\xb8\xd0\xb7\xd0\xb2\xd0\xb5\xd1\x81\xd1\x82\xd0\xbd\xd0\xb0\xd1\x8f \xd0\xba\xd0\xb0\xd0\xba \xd0\xa2\xd0\xb8\xd1\x84\xd0\xbe\xd0\xb7\xd0\xbd\xd0\xb0\xd1\x8f \xd0\x9c\xd1\x8d\xd1\x80\xd0\xb8, \xd1\x8f\xd0\xb2\xd0\xbb\xd1\x8f\xd0\xbb\xd0\xb0\xd1\x81\xd1\x8c \xd1\x80\xd0\xb0\xd0\xb7\xd0\xbd\xd0\xbe\xd1\x81\xd1\x87\xd0\xb8\xd0\xba\xd0\xbe\xd0\xbc \xd0\xb1\xd0\xbe\xd0\xbb\xd0\xb5\xd0\xb7\xd0\xbd\xd0\xb8 \xd0\xb8 \xd0\xb1\xd1\x8b\xd0\xbb\xd0\xb0 \xd0\xbf\xd0\xbe\xd0\xb2\xd0\xb8\xd0\xbd\xd0\xbd\xd0\xb0 \xd0\xb2 \xd0\xbd\xd0\xb5\xd1\x81\xd0\xba\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd0\xb8\xd1\x85 \xd0\xb2\xd1\x81\xd0\xbf\xd1\x8b\xd1\x88\xd0\xba\xd0\xb0\xd1\x85 \xd0\xb1\xd1\x80\xd1\x8e\xd1\x88\xd0\xbd\xd0\xbe\xd0\xb3\xd0\xbe \xd1\x82\xd0\xb8\xd1\x84\xd0\xb0. \xd0\x9d\xd0\xb0 \xd1\x81\xd0\xb0\xd0\xbc\xd0\xbe\xd0\xbc \xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb5 \xd0\xbf\xd0\xbe\xd1\x85\xd0\xbe\xd0\xb6\xd0\xb8\xd1\x85 \xd1\x81\xd0\xbb\xd1\x83\xd1\x87\xd0\xb0\xd0\xb5\xd0\xb2 \xd0\xb1\xd1\x8b\xd0\xbb\xd0\xbe \xd0\xbd\xd0\xb5\xd1\x81\xd0\xba\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd0\xbe. \xd0\x9d\xd0\xb0\xd0\xbf\xd1\x80\xd0\xb8\xd0\xbc\xd0\xb5\xd1\x80, \xd1\x80\xd0\xb5\xd1\x81\xd1\x82\xd0\xbe\xd1\x80\xd0\xb0\xd1\x82\xd0\xbe\xd1\x80 \xd0\x90\xd0\xbb\xd1\x8c\xd1\x84\xd0\xbe\xd0\xbd\xd1\x81 \xd0\x9a\xd0\xbe\xd1\x82\xd0\xb8\xd0\xbb\xd1\x81 \xd1\x82\xd0\xbe\xd0\xb6\xd0\xb5 \xd0\xb1\xd1\x8b\xd0\xbb \xd1\x80\xd0\xb0\xd0\xb7\xd0\xbd\xd0\xbe\xd1\x81\xd1\x87\xd0\xb8\xd0\xba\xd0\xbe\xd0\xbc \xd1\x82\xd0\xb8\xd1\x84\xd0\xb0, \xd0\xbe\xd0\xb4\xd0\xbd\xd0\xb0\xd0\xba\xd0\xbe \xd0\xb2 \xd0\xbe\xd1\x82\xd0\xbb\xd0\xb8\xd1\x87\xd0\xb8\xd0\xb5 \xd0\xbe\xd1\x82 \xd0\x9c\xd0\xb0\xd0\xbb\xd0\xbb\xd0\xbe\xd0\xbd \xd1\x81\xd0\xbe\xd0\xb3\xd0\xbb\xd0\xb0\xd1\x81\xd0\xb8\xd0\xbb\xd1\x81\xd1\x8f \xd1\x81\xd0\xbe\xd1\x82\xd1\x80\xd1\x83\xd0\xb4\xd0\xbd\xd0\xb8\xd1\x87\xd0\xb0\xd1\x82\xd1\x8c \xd1\x81 \xd0\xb2\xd0\xbb\xd0\xb0\xd1\x81\xd1\x82\xd1\x8f\xd0\xbc\xd0\xb8. \xd0\x9f\xd0\xbe \xd0\xb0\xd0\xbd\xd0\xb0\xd0\xbb\xd0\xbe\xd0\xb3\xd0\xb8\xd0\xb8 \xd1\x81 \xd0\xa2\xd0\xb8\xd1\x84\xd0\xbe\xd0\xb7\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\x9c\xd1\x8d\xd1\x80\xd0\xb8 \xd1\x80\xd0\xb5\xd1\x81\xd1\x82\xd0\xbe\xd1\x80\xd0\xb0\xd1\x82\xd0\xbe\xd1\x80\xd0\xb0 \xd0\x90\xd0\xbb\xd1\x8c\xd1\x84\xd0\xbe\xd0\xbd\xd1\x81\xd0\xb0 \xd0\x9a\xd0\xbe\xd1\x82\xd0\xb8\xd0\xbb\xd1\x81\xd0\xb0 \xd0\xbd\xd0\xb0\xd0\xb7\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x8e\xd1\x82 \xc2\xab\xd0\xa2\xd0\xb8\xd1\x84\xd0\xbe\xd0\xb7\xd0\xbd\xd1\x8b\xd0\xbc \xd0\x90\xd0\xbb\xd1\x8c\xd1\x84\xd0\xbe\xd0\xbd\xd1\x81\xd0\xbe\xd0\xbc\xc2\xbb.    \n\n4. \xd1\x84\xd0\xb0\xd1\x80\xd1\x84\xd0\xbe\xd1\x80\xd0\xb0\n\n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb8\xd0\xb7\xd0\xb2\xd0\xbe\xd0\xb4\xd0\xb8\xd1\x82\xd0\xb5\xd0\xbb\xd0\xb8 \xd0\xbf\xd1\x83\xd0\xb4\xd1\x80\xd1\x8b \xd0\xb4\xd0\xbb\xd1\x8f \xd0\xbf\xd0\xb0\xd1\x80\xd0\xb8\xd0\xba\xd0\xbe\xd0\xb2 \xd0\xbd\xd0\xb5 \xd1\x85\xd0\xbe\xd1\x82\xd0\xb5\xd0\xbb\xd0\xb8, \xd1\x87\xd1\x82\xd0\xbe\xd0\xb1\xd1\x8b \xd0\xb2\xd1\x81\xd0\xba\xd1\x80\xd1\x8b\xd0\xbb\xd0\xbe\xd1\x81\xd1\x8c \xd0\xb8\xd1\x81\xd0\xbf\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xb7\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xb2\xd0\xbc\xd0\xb5\xd1\x81\xd1\x82\xd0\xbe \xd0\xbc\xd1\x83\xd0\xba\xd0\xb8 \xd0\xb1\xd0\xb5\xd0\xbb\xd0\xbe\xd0\xb9 \xd0\xb3\xd0\xbb\xd0\xb8\xd0\xbd\xd1\x8b, \xd1\x81\xd1\x87\xd0\xb8\xd1\x82\xd0\xb0\xd0\xb2\xd1\x88\xd0\xb5\xd0\xb9\xd1\x81\xd1\x8f \xd0\xb4\xd0\xb5\xd1\x88\xd1\x91\xd0\xb2\xd0\xbe\xd0\xb9. \xd0\x9e\xd0\xb4\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\xb8\xd0\xb7 \xd1\x82\xd1\x80\xd1\x83\xd0\xb4\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd0\xb5\xd0\xb9 \xd0\xb2 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb8\xd0\xb7\xd0\xb2\xd0\xbe\xd0\xb4\xd1\x81\xd1\x82\xd0\xb2\xd0\xb5 \xd1\x84\xd0\xb0\xd1\x80\xd1\x84\xd0\xbe\xd1\x80\xd0\xb0 \xd0\xb2 \xd0\x95\xd0\xb2\xd1\x80\xd0\xbe\xd0\xbf\xd0\xb5 \xd1\x8f\xd0\xb2\xd0\xbb\xd1\x8f\xd0\xbb\xd0\xbe\xd1\x81\xd1\x8c \xd0\xbe\xd1\x82\xd1\x81\xd1\x83\xd1\x82\xd1\x81\xd1\x82\xd0\xb2\xd0\xb8\xd0\xb5 \xd1\x81\xd1\x8b\xd1\x80\xd1\x8c\xd1\x8f \xd0\xbd\xd0\xb0\xd0\xb4\xd0\xbb\xd0\xb5\xd0\xb6\xd0\xb0\xd1\x89\xd0\xb5\xd0\xb3\xd0\xbe \xd0\xba\xd0\xb0\xd1\x87\xd0\xb5\xd1\x81\xd1\x82\xd0\xb2\xd0\xb0. \xd0\x9d\xd0\xb0 \xd1\x81\xd0\xb0\xd0\xbc\xd0\xbe\xd0\xbc \xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb5 \xd0\xb8\xd1\x81\xd1\x82\xd0\xbe\xd1\x87\xd0\xbd\xd0\xb8\xd0\xba\xd0\xb8 \xd0\xbf\xd0\xbe\xd0\xb4\xd1\x85\xd0\xbe\xd0\xb4\xd1\x8f\xd1\x89\xd0\xb5\xd0\xb9 \xd0\xb3\xd0\xbb\xd0\xb8\xd0\xbd\xd1\x8b \xd1\x83\xd0\xb6\xd0\xb5 \xd0\xb1\xd1\x8b\xd0\xbb\xd0\xb8 \xd0\xb8\xd0\xb7\xd0\xb2\xd0\xb5\xd1\x81\xd1\x82\xd0\xbd\xd1\x8b \xd0\xb8 \xd1\x80\xd0\xb0\xd0\xb7\xd1\x80\xd0\xb0\xd0\xb1\xd0\xb0\xd1\x82\xd1\x8b\xd0\xb2\xd0\xb0\xd0\xbb\xd0\xb8\xd1\x81\xd1\x8c, \xd0\xbd\xd0\xbe \xd0\xb8\xd1\x85 \xd1\x81\xd1\x83\xd1\x89\xd0\xb5\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xbd\xd0\xb5 \xd0\xb0\xd1\x84\xd0\xb8\xd1\x88\xd0\xb8\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbb\xd0\xbe\xd1\x81\xd1\x8c.\n\n5. \xd0\xad\xd0\xbb\xd1\x8c\xd1\x84\xd0\xb8\xd0\xbd\xd0\xb8\xd1\x82\xd0\xb8\xd0\xb2\n\n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\xbc\xd1\x8b \xd0\xbf\xd1\x80\xd0\xbe\xd1\x86\xd0\xb8\xd1\x82\xd0\xb8\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbb\xd0\xb8 \xd0\xb2\xd0\xb0\xd0\xbc \xd0\xbd\xd0\xb0\xd1\x87\xd0\xb0\xd0\xbb\xd0\xbe \xd0\xbf\xd0\xbe\xd1\x8d\xd0\xbc\xd1\x8b \xc2\xab\xd0\x9d\xd0\xb0\xd0\xbc\xd0\xb0\xd1\x80\xd0\xb8\xd1\x8d\xc2\xbb (\xd1\x82\xd0\xb0\xd0\xba\xd0\xb6\xd0\xb5 \xd0\xb8\xd0\xb7\xd0\xb2\xd0\xb5\xd1\x81\xd1\x82\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\xba\xd0\xb0\xd0\xba \xc2\xab\xd0\x9f\xd0\xb5\xd1\x87\xd0\xb0\xd0\xbb\xd1\x8c \xd0\x93\xd0\xb0\xd0\xbb\xd0\xb0\xd0\xb4\xd1\x80\xd0\xb8\xd1\x8d\xd0\xbb\xd1\x8c\xc2\xbb), \xd0\xb2 \xd0\xbe\xd1\x80\xd0\xb8\xd0\xb3\xd0\xb8\xd0\xbd\xd0\xb0\xd0\xbb\xd0\xb5 \xd0\xbd\xd0\xb0\xd0\xbf\xd0\xb8\xd1\x81\xd0\xb0\xd0\xbd\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\xbd\xd0\xb0 \xd0\xba\xd0\xb2\xd0\xb5\xd0\xbd\xd1\x8c\xd0\xb5 \xe2\x80\x93 \xd1\x8d\xd0\xbb\xd1\x8c\xd1\x84\xd0\xb8\xd0\xb9\xd1\x81\xd0\xba\xd0\xbe\xd0\xbc \xd1\x8f\xd0\xb7\xd1\x8b\xd0\xba\xd0\xb5, \xd1\x81\xd0\xbe\xd0\xb7\xd0\xb4\xd0\xb0\xd0\xbd\xd0\xbd\xd0\xbe\xd0\xbc \xd0\xbf\xd1\x80\xd0\xbe\xd1\x84\xd0\xb5\xd1\x81\xd1\x81\xd0\xbe\xd1\x80\xd0\xbe\xd0\xbc \xd0\xa2\xd0\xbe\xd0\xbb\xd0\xba\xd0\xb8\xd0\xbd\xd0\xbe\xd0\xbc. \xd0\x92 \xd1\x8d\xd1\x82\xd0\xbe\xd0\xbc \xd1\x8f\xd0\xb7\xd1\x8b\xd0\xba\xd0\xb5 10 \xd0\xbf\xd0\xb0\xd0\xb4\xd0\xb5\xd0\xb6\xd0\xb5\xd0\xb9, \xd0\xb2 \xd1\x82\xd0\xbe\xd0\xbc \xd1\x87\xd0\xb8\xd1\x81\xd0\xbb\xd0\xb5 \xd1\x82\xd0\xb0\xd0\xba \xd0\xbd\xd0\xb0\xd0\xb7\xd1\x8b\xd0\xb2\xd0\xb0\xd0\xb5\xd0\xbc\xd1\x8b\xd0\xb9 \xc2\xab\xd1\x8d\xd0\xbb\xd1\x8c\xd1\x84\xd0\xb8\xd0\xbd\xd0\xb8\xd1\x82\xd0\xb8\xd0\xb2\xc2\xbb. \xd0\x92 \xd1\x81\xd0\xbe\xd0\xb7\xd0\xb4\xd0\xb0\xd0\xbd\xd0\xbd\xd0\xbe\xd0\xbc \xd0\xa2\xd0\xbe\xd0\xbb\xd0\xba\xd0\xb8\xd0\xbd\xd0\xbe\xd0\xbc \xd0\xbc\xd0\xb8\xd1\x80\xd0\xb5 \xd1\x8d\xd0\xbb\xd1\x8c\xd1\x84\xd1\x8b \xd0\xb6\xd0\xb8\xd0\xb2\xd1\x83\xd1\x82 \xd1\x87\xd1\x80\xd0\xb5\xd0\xb7\xd0\xb2\xd1\x8b\xd1\x87\xd0\xb0\xd0\xb9\xd0\xbd\xd0\xbe \xd0\xb4\xd0\xbe\xd0\xbb\xd0\xb3\xd0\xbe \xd0\xb8 \xd0\xbd\xd0\xb5\xd0\xb6\xd0\xbd\xd0\xbe \xd0\xbb\xd1\x8e\xd0\xb1\xd1\x8f\xd1\x82 \xd0\xbb\xd0\xb5\xd1\x81\xd0\xb0 \xd0\xb8 \xd0\xbf\xd0\xbe\xd1\x8d\xd0\xb7\xd0\xb8\xd1\x8e.\n\n6. \xd0\xbd\xd0\xbe\xd1\x81\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb3\xd0\xbe\xd0\xb2\n\n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\xa1\xd0\xb0\xd0\xbc\xd0\xbe\xd0\xb5 \xd0\xb7\xd0\xb0\xd0\xbc\xd0\xb5\xd1\x82\xd0\xbd\xd0\xbe\xd0\xb5 \xd0\xb2\xd0\xbd\xd0\xb5\xd1\x88\xd0\xbd\xd0\xb5\xd0\xb5 \xd0\xbe\xd1\x82\xd0\xbb\xd0\xb8\xd1\x87\xd0\xb8\xd0\xb5 \xd1\x81\xd0\xb0\xd0\xbc\xd1\x86\xd0\xbe\xd0\xb2 \xe2\x80\x93 \xd1\x81\xd0\xb0\xd0\xb9\xd0\xb3\xd0\xb0\xd0\xba\xd0\xbe\xd0\xb2 \xe2\x80\x93 \xd0\xbe\xd1\x82 \xd1\x81\xd0\xb0\xd0\xbc\xd0\xbe\xd0\xba \xe2\x80\x93 \xd0\xbd\xd0\xb0\xd0\xbb\xd0\xb8\xd1\x87\xd0\xb8\xd0\xb5 \xd1\x80\xd0\xbe\xd0\xb3\xd0\xbe\xd0\xb2, \xd0\xb7\xd0\xb0 \xd0\xba\xd0\xbe\xd1\x82\xd0\xbe\xd1\x80\xd1\x8b\xd0\xbc\xd0\xb8 \xd0\xb8\xd0\xb7-\xd0\xb7\xd0\xb0 \xd1\x8d\xd1\x82\xd0\xbe\xd0\xb9 \xd0\xba\xd0\xb0\xd0\xbc\xd0\xbf\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb8 \xd0\xb1\xd1\x80\xd0\xb0\xd0\xba\xd0\xbe\xd0\xbd\xd1\x8c\xd0\xb5\xd1\x80\xd1\x8b \xd1\x81\xd1\x82\xd0\xb0\xd0\xbb\xd0\xb8 \xd0\xbe\xd1\x85\xd0\xbe\xd1\x82\xd0\xb8\xd1\x82\xd1\x8c\xd1\x81\xd1\x8f \xd0\xb3\xd0\xbe\xd1\x80\xd0\xb0\xd0\xb7\xd0\xb4\xd0\xbe \xd0\xb0\xd0\xba\xd1\x82\xd0\xb8\xd0\xb2\xd0\xbd\xd0\xb5\xd0\xb5. \xd0\xa1\xd0\xbf\xd1\x80\xd0\xbe\xd1\x81 \xd0\xbd\xd0\xb0 \xd1\x80\xd0\xbe\xd0\xb3\xd0\xb0 \xd1\x81\xd0\xb0\xd0\xb9\xd0\xb3\xd0\xb0\xd0\xba\xd0\xb0 \xd0\xb2\xd1\x8b\xd1\x80\xd0\xbe\xd1\x81 \xd0\xb8\xd0\xb7-\xd0\xb7\xd0\xb0 \xd1\x82\xd0\xbe\xd0\xb3\xd0\xbe, \xd1\x87\xd1\x82\xd0\xbe \xd1\x8d\xd0\xba\xd0\xbe\xd0\xbb\xd0\xbe\xd0\xb3\xd0\xb8 \xd1\x86\xd0\xb5\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xb0\xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xbd\xd0\xbe \xd0\xbf\xd1\x80\xd0\xbe\xd0\xbf\xd0\xb0\xd0\xb3\xd0\xb0\xd0\xbd\xd0\xb4\xd0\xb8\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbb\xd0\xb8 \xd1\x81\xd1\x80\xd0\xb5\xd0\xb4\xd0\xb8 \xd1\x81\xd0\xbf\xd0\xb5\xd1\x86\xd0\xb8\xd0\xb0\xd0\xbb\xd0\xb8\xd1\x81\xd1\x82\xd0\xbe\xd0\xb2 \xd0\xbf\xd0\xbe \xd0\xb2\xd0\xbe\xd1\x81\xd1\x82\xd0\xbe\xd1\x87\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\xbc\xd0\xb5\xd0\xb4\xd0\xb8\xd1\x86\xd0\xb8\xd0\xbd\xd0\xb5 \xd0\xb8\xd1\x81\xd0\xbf\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xb7\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5 \xd1\x80\xd0\xbe\xd0\xb3\xd0\xbe\xd0\xb2 \xd1\x81\xd0\xb0\xd0\xb9\xd0\xb3\xd0\xb0\xd0\xba\xd0\xbe\xd0\xb2 \xd0\xb2\xd0\xbc\xd0\xb5\xd1\x81\xd1\x82\xd0\xbe \xd1\x80\xd0\xbe\xd0\xb3\xd0\xb0 \xd0\xbd\xd0\xbe\xd1\x81\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb3\xd0\xb0.\n\n7. HAL-9000\n    \n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbf\xd1\x8c\xd1\x8e\xd1\x82\xd0\xb5\xd1\x80, \xd1\x83\xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd0\xbb\xd1\x8f\xd1\x8e\xd1\x89\xd0\xb8\xd0\xb9 \xd0\xbf\xd0\xbe\xd0\xbb\xd1\x91\xd1\x82\xd0\xbe\xd0\xbc \xd0\xba\xd0\xbe\xd1\x80\xd0\xb0\xd0\xb1\xd0\xbb\xd1\x8f \xd0\xb2 \xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c\xd0\xbc\xd0\xb5 \xc2\xab\xd0\x9a\xd0\xbe\xd1\x81\xd0\xbc\xd0\xb8\xd1\x87\xd0\xb5\xd1\x81\xd0\xba\xd0\xb0\xd1\x8f \xd0\x9e\xd0\xb4\xd0\xb8\xd1\x81\xd1\x81\xd0\xb5\xd1\x8f \xe2\x80\x93 2001\xc2\xbb, \xd0\xbf\xd1\x80\xd0\xb8\xd0\xbd\xd0\xb8\xd0\xbc\xd0\xb0\xd0\xb5\xd1\x82 \xd1\x80\xd0\xb5\xd1\x88\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xbe\xd0\xb1\xd0\xbc\xd0\xb0\xd0\xbd\xd1\x83\xd1\x82\xd1\x8c \xd0\xb8 \xd1\x83\xd0\xb1\xd0\xb8\xd1\x82\xd1\x8c \xd1\x8d\xd0\xba\xd0\xb8\xd0\xbf\xd0\xb0\xd0\xb6. \xd0\x9a\xd1\x83\xd0\xb1\xd1\x80\xd0\xb8\xd0\xba \xd1\x81\xd0\xb0\xd0\xbc \xd0\xb1\xd1\x8b\xd0\xbb \xd0\xbe\xd0\xbf\xd1\x8b\xd1\x82\xd0\xbd\xd1\x8b\xd0\xbc \xd1\x88\xd0\xb0\xd1\x85\xd0\xbc\xd0\xb0\xd1\x82\xd0\xb8\xd1\x81\xd1\x82\xd0\xbe\xd0\xbc, \xd0\xba \xd0\xbf\xd1\x80\xd0\xb8\xd0\xbc\xd0\xb5\xd1\x80\xd1\x83, \xd1\x84\xd0\xb0\xd0\xbc\xd0\xb8\xd0\xbb\xd0\xb8\xd1\x8f \xd0\xbe\xd0\xb4\xd0\xbd\xd0\xbe\xd0\xb3\xd0\xbe \xd0\xb8\xd0\xb7 \xd0\xbf\xd0\xb5\xd1\x80\xd1\x81\xd0\xbe\xd0\xbd\xd0\xb0\xd0\xb6\xd0\xb5\xd0\xb9 \xe2\x80\x94 \xd0\xa1\xd0\xbc\xd1\x8b\xd1\x81\xd0\xbb\xd0\xbe\xd0\xb2. \xd0\x9f\xd0\xbe \xd0\xbc\xd0\xbd\xd0\xb5\xd0\xbd\xd0\xb8\xd1\x8e \xd0\xa1\xd0\xbc\xd0\xb8\xd1\x82\xd0\xb0, \xd1\x82\xd0\xbe, \xd1\x87\xd1\x82\xd0\xbe \xd0\xba\xd0\xbe\xd0\xbc\xd0\xbf\xd1\x8c\xd1\x8e\xd1\x82\xd0\xb5\xd1\x80 \xd0\xbd\xd0\xb0\xd1\x87\xd0\xb0\xd0\xbb \xd0\xbe\xd0\xb1\xd0\xbc\xd0\xb0\xd0\xbd\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c \xd1\x8d\xd0\xba\xd0\xb8\xd0\xbf\xd0\xb0\xd0\xb6 \xd1\x83\xd0\xb6\xd0\xb5 \xd0\xb2\xd0\xbe \xd0\xb2\xd1\x80\xd0\xb5\xd0\xbc\xd1\x8f \xd1\x88\xd0\xb0\xd1\x85\xd0\xbc\xd0\xb0\xd1\x82\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\xbf\xd0\xb0\xd1\x80\xd1\x82\xd0\xb8\xd0\xb8 \xd0\xbd\xd0\xb5 \xd0\xbc\xd0\xbe\xd0\xb3\xd0\xbb\xd0\xbe \xd0\xb1\xd1\x8b\xd1\x82\xd1\x8c \xd1\x81\xd0\xbb\xd1\x83\xd1\x87\xd0\xb0\xd0\xb9\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd1\x8c\xd1\x8e.\n\n8. \xd1\x83\xd0\xb4\xd0\xb0\xd0\xb2\xd0\xb0\n    \n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\xbd\xd0\xb5 \xd1\x82\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd0\xbe \xd1\x83 \xd0\xbb\xd1\x8e\xd0\xb4\xd0\xb5\xd0\xb9 \xd0\xb1\xd0\xb0\xd0\xb1\xd1\x83\xd1\x88\xd0\xba\xd0\xb8 \xd0\xbf\xd0\xbe\xd0\xbc\xd0\xbe\xd0\xb3\xd0\xb0\xd1\x8e\xd1\x82 \xd0\xb2\xd0\xbe\xd1\x81\xd0\xbf\xd0\xb8\xd1\x82\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c \xd0\xbf\xd0\xbe\xd1\x82\xd0\xbe\xd0\xbc\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe. \xd0\x93\xd0\xbe\xd0\xbb\xd1\x83\xd0\xb1\xd0\xb0\xd1\x8f \xd1\x82\xd0\xba\xd0\xb0\xd0\xbd\xd1\x8c \xd1\x81 \xd1\x80\xd0\xbe\xd0\xbc\xd0\xb0\xd1\x88\xd0\xba\xd0\xb0\xd0\xbc\xd0\xb8 \xd0\xbf\xd0\xbe\xd1\x88\xd0\xbb\xd0\xb0 \xd0\xbd\xd0\xb0 \xd0\xb8\xd0\xb7\xd0\xb3\xd0\xbe\xd1\x82\xd0\xbe\xd0\xb2\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xb1\xd0\xb0\xd0\xb1\xd1\x83\xd1\x88\xd0\xba\xd0\xb8 \xd1\x83\xd0\xb4\xd0\xb0\xd0\xb2\xd0\xb0 \xd0\xb8\xd0\xb7 \xd0\xb8\xd0\xb7\xd0\xb2\xd0\xb5\xd1\x81\xd1\x82\xd0\xbd\xd0\xbe\xd0\xb3\xd0\xbe \xd0\xba\xd1\x83\xd0\xba\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xbe\xd0\xb3\xd0\xbe \xd0\xbc\xd1\x83\xd0\xbb\xd1\x8c\xd1\x82\xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c\xd0\xbc\xd0\xb0.\n\n9. \xc2\xab\xd0\x9e\xd0\xb3\xd0\xbe\xd0\xbd\xd1\x91\xd0\xba\xc2\xbb\n\n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\x9e\xd0\x9d \xe2\x80\x94 \xc2\xab\xd0\x9e\xd0\xb3\xd0\xbe\xd0\xbd\xd1\x91\xd0\xba\xc2\xbb, \xd0\xbf\xd0\xbe\xd0\xbf\xd1\x83\xd0\xbb\xd1\x8f\xd1\x80\xd0\xbd\xd1\x8b\xd0\xb9 \xd0\xb6\xd1\x83\xd1\x80\xd0\xbd\xd0\xb0\xd0\xbb, \xd1\x81\xd1\x83\xd1\x89\xd0\xb5\xd1\x81\xd1\x82\xd0\xb2\xd1\x83\xd1\x8e\xd1\x89\xd0\xb8\xd0\xb9 \xd0\xb4\xd0\xbe\xd0\xbd\xd1\x8b\xd0\xbd\xd0\xb5. \xd0\x92\xd0\xbe\xd0\xbe\xd0\xb1\xd1\x89\xd0\xb5-\xd1\x82\xd0\xbe \xd0\xb2\xd1\x8b\xd0\xbf\xd1\x83\xd1\x81\xd0\xba \xd0\xb7\xd0\xb0\xd0\xb6\xd0\xb8\xd0\xb3\xd0\xb0\xd0\xbb\xd0\xbe\xd0\xba \xd0\xbc\xd0\xbe\xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb8 \xc2\xab\xd0\x9e\xd0\xb3\xd0\xbe\xd0\xbd\xd1\x91\xd0\xba\xc2\xbb \xd0\xb1\xd1\x8b\xd0\xbb \xd0\xbf\xd1\x80\xd0\xb8\xd1\x83\xd1\x80\xd0\xbe\xd1\x87\xd0\xb5\xd0\xbd \xd0\xba \xd0\xbf\xd0\xb5\xd1\x80\xd0\xb2\xd1\x8b\xd0\xbc \xd0\xbf\xd0\xbe\xd0\xbb\xd1\x91\xd1\x82\xd0\xb0\xd0\xbc \xd0\xb2 \xd0\xba\xd0\xbe\xd1\x81\xd0\xbc\xd0\xbe\xd1\x81, \xd0\xbd\xd0\xbe, \xd0\xb2\xd0\xbe\xd0\xb7\xd0\xbc\xd0\xbe\xd0\xb6\xd0\xbd\xd0\xbe, \xd0\xb8\xd0\xbd\xd0\xb8\xd1\x86\xd0\xb8\xd0\xb0\xd1\x82\xd0\xb8\xd0\xb2\xd0\xb0 \xd0\xb6\xd1\x83\xd1\x80\xd0\xbd\xd0\xb0\xd0\xbb\xd0\xb0 \xd1\x82\xd0\xbe\xd0\xb6\xd0\xb5 \xd1\x81\xd1\x8b\xd0\xb3\xd1\x80\xd0\xb0\xd0\xbb\xd0\xb0 \xd1\x81\xd0\xb2\xd0\xbe\xd1\x8e \xd1\x80\xd0\xbe\xd0\xbb\xd1\x8c.\n\n10. \xd0\xa1\xd0\xb5\xd1\x80\xd0\xb3\xd0\xb5\xd0\xb9 \xd0\x9f\xd0\xb0\xd0\xb2\xd0\xbb\xd0\xbe\xd0\xb2\xd0\xb8\xd1\x87 \xd0\x9a\xd0\xbe\xd1\x80\xd0\xbe\xd0\xbb\xd0\xb5\xd0\xb2\n\n    \xd0\x9a\xd0\xbe\xd0\xbc\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd0\xb0\xd1\x80\xd0\xb8\xd0\xb9: \xd0\x90\xd0\xb7\xd0\xb8\xd0\xbc\xd0\xbe\xd0\xb2 \xd0\xbf\xd0\xb8\xd1\x81\xd0\xb0\xd0\xbb: \xc2\xab\xd0\x9d\xd0\xb5\xd0\xba\xd0\xbe\xd1\x82\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb5 \xd0\xb1\xd0\xb5\xd1\x81\xd0\xbf\xd0\xbe\xd0\xba\xd0\xbe\xd0\xb9\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe \xd0\xb2\xd1\x8b\xd0\xb7\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x8e\xd1\x82 \xd0\xb4\xd0\xbe\xd0\xb3\xd0\xb0\xd0\xb4\xd0\xba\xd0\xb8 \xd0\xbe \xd1\x82\xd0\xbe\xd0\xbc, \xd1\x87\xd1\x82\xd0\xbe \xd0\xba\xd0\xb0\xd0\xba \xd0\xbc\xd0\xb8\xd0\xbd\xd0\xb8\xd0\xbc\xd1\x83\xd0\xbc \xd1\x87\xd0\xb0\xd1\x81\xd1\x82\xd1\x8c \xd0\xbb\xd1\x83\xd0\xbd\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\xbf\xd0\xbe\xd0\xb2\xd0\xb5\xd1\x80\xd1\x85\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd0\xb8 \xd0\xbf\xd0\xbe\xd0\xba\xd1\x80\xd1\x8b\xd1\x82\xd0\xb0 \xd1\x82\xd0\xbe\xd0\xbb\xd1\x81\xd1\x82\xd1\x8b\xd0\xbc \xd1\x81\xd0\xbb\xd0\xbe\xd0\xb5\xd0\xbc \xd0\xbf\xd1\x8b\xd0\xbb\xd0\xb8. \xd0\xa4\xd0\xbe\xd1\x82\xd0\xbe\xd0\xb3\xd1\x80\xd0\xb0\xd1\x84\xd0\xb8\xd0\xb8, \xd1\x81\xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb0\xd0\xbd\xd0\xbd\xd1\x8b\xd0\xb5 \xd1\x81 \xd0\xb1\xd0\xbb\xd0\xb8\xd0\xb7\xd0\xba\xd0\xbe\xd0\xb3\xd0\xbe \xd1\x80\xd0\xb0\xd1\x81\xd1\x81\xd1\x82\xd0\xbe\xd1\x8f\xd0\xbd\xd0\xb8\xd1\x8f, \xd0\xbd\xd0\xb8\xd1\x87\xd0\xb5\xd0\xb3\xd0\xbe \xd0\xbf\xd0\xbe\xd0\xb4\xd0\xbe\xd0\xb1\xd0\xbd\xd0\xbe\xd0\xb3\xd0\xbe \xd0\xbd\xd0\xb5 \xd0\xbf\xd0\xbe\xd0\xba\xd0\xb0\xd0\xb7\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x8e\xd1\x82, \xd0\xbd\xd0\xbe \xd0\xb2\xd0\xb5\xd1\x80\xd0\xbe\xd1\x8f\xd1\x82\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd1\x8c \xd0\xbd\xd0\xb5 \xd0\xb8\xd1\x81\xd0\xba\xd0\xbb\xd1\x8e\xd1\x87\xd0\xb5\xd0\xbd\xd0\xb0. \xd0\x95\xd1\x81\xd0\xbb\xd0\xb8 \xd1\x8d\xd1\x82\xd0\xbe \xd1\x82\xd0\xb0\xd0\xba, \xd1\x82\xd0\xbe \xd0\xbf\xd0\xb5\xd1\x80\xd0\xb5\xd0\xbc\xd0\xb5\xd1\x89\xd0\xb0\xd1\x82\xd1\x8c\xd1\x81\xd1\x8f \xd0\xbf\xd0\xbe \xd0\x9b\xd1\x83\xd0\xbd\xd0\xb5 \xd0\xbc\xd0\xbe\xd0\xb6\xd0\xbd\xd0\xbe \xd0\xb1\xd1\x83\xd0\xb4\xd0\xb5\xd1\x82 \xd1\x82\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd0\xbe \xd0\xbd\xd0\xb0 \xd1\x83\xd1\x81\xd1\x82\xd1\x80\xd0\xbe\xd0\xb9\xd1\x81\xd1\x82\xd0\xb2\xd0\xb0\xd1\x85 \xd0\xb2\xd1\x80\xd0\xbe\xd0\xb4\xd0\xb5 \xd0\xbc\xd0\xbe\xd1\x82\xd0\xbe\xd1\x81\xd0\xb0\xd0\xbd\xd0\xb5\xd0\xb9 \xe2\x80\x94 \xd0\xbd\xd0\xbe \xd0\xb2\xd1\x81\xd0\xb5 \xd1\x80\xd0\xb0\xd0\xb2\xd0\xbd\xd0\xbe \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb1\xd0\xbb\xd0\xb5\xd0\xbc\xd0\xb0 \xd1\x80\xd0\xb5\xd1\x88\xd0\xb0\xd0\xb5\xd0\xbc\xd0\xb0\xc2\xbb [1].\n    \xd0\xad\xd1\x82\xd0\xb8 \xd0\xb6\xd0\xb5 \xd0\xbe\xd0\xbf\xd0\xb0\xd1\x81\xd0\xb5\xd0\xbd\xd0\xb8\xd1\x8f \xd0\xb2\xd1\x8b\xd1\x81\xd0\xba\xd0\xb0\xd0\xb7\xd1\x8b\xd0\xb2\xd0\xb0\xd0\xbb\xd0\xb8\xd1\x81\xd1\x8c \xd0\xb2 \xd0\xa1\xd0\xa1\xd0\xa1\xd0\xa0 \xd0\xbf\xd1\x80\xd0\xb8 \xd0\xbf\xd0\xbb\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb8 \xd0\xbc\xd1\x8f\xd0\xb3\xd0\xba\xd0\xbe\xd0\xb9 \xd0\xbf\xd0\xbe\xd1\x81\xd0\xb0\xd0\xb4\xd0\xba\xd0\xb8 \xd0\xbd\xd0\xb0 \xd0\x9b\xd1\x83\xd0\xbd\xd1\x83. \xd0\xa1\xd0\xbe\xd0\xb3\xd0\xbb\xd0\xb0\xd1\x81\xd0\xbd\xd0\xbe \xd0\xbf\xd0\xbe\xd0\xbf\xd1\x83\xd0\xbb\xd1\x8f\xd1\x80\xd0\xbd\xd0\xbe\xd0\xb9 \xd0\xbb\xd0\xb5\xd0\xb3\xd0\xb5\xd0\xbd\xd0\xb4\xd0\xb5, \xd0\x9a\xd0\xbe\xd1\x80\xd0\xbe\xd0\xbb\xd0\xb5\xd0\xb2 \xd0\xbe\xd1\x82\xd0\xba\xd0\xbb\xd0\xbe\xd0\xbd\xd0\xb8\xd0\xbb \xd1\x8d\xd1\x82\xd0\xb8 \xd0\xbe\xd0\xbf\xd0\xb0\xd1\x81\xd0\xb5\xd0\xbd\xd0\xb8\xd1\x8f, \xd0\xb2\xd1\x8b\xd0\xb4\xd0\xb0\xd0\xb2 \xd1\x81\xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd0\xba\xd1\x83: \xc2\xab\xd0\x9b\xd1\x83\xd0\xbd\xd0\xb0 \xd1\x82\xd0\xb2\xd0\xb5\xd1\x80\xd0\xb4\xd0\xb0\xd1\x8f\xc2\xbb. \xd0\x9e\xd0\xb4\xd0\xbd\xd0\xb0\xd0\xba\xd0\xbe \xd1\x80\xd0\xb5\xd0\xb0\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xbe \xd1\x81\xd0\xbe\xd1\x85\xd1\x80\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb2\xd1\x88\xd0\xb8\xd0\xb9\xd1\x81\xd1\x8f \xd0\xb0\xd0\xb2\xd1\x82\xd0\xbe\xd0\xb3\xd1\x80\xd0\xb0\xd1\x84 \xd1\x81\xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd0\xba\xd0\xb8 \xd0\xb3\xd0\xbb\xd0\xb0\xd1\x81\xd0\xb8\xd1\x82 (\xd0\xb7\xd0\xb4\xd0\xb5\xd1\x81\xd1\x8c \xd0\x9b\xd0\x9a \xe2\x80\x93 \xd0\xbb\xd1\x83\xd0\xbd\xd0\xbd\xd1\x8b\xd0\xb9 \xd0\xba\xd0\xbe\xd1\x80\xd0\xb0\xd0\xb1\xd0\xbb\xd1\x8c):\n    \xd0\xa1\xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd0\xba\xd0\xb0 .\n    \xd0\x9f\xd0\xbe\xd1\x81\xd0\xb0\xd0\xb4\xd0\xba\xd1\x83 \xd0\x9b\xd0\x9a \xd1\x81\xd0\xbb\xd0\xb5\xd0\xb4\xd1\x83\xd0\xb5\xd1\x82 \xd1\x80\xd0\xb0\xd1\x81\xd1\x87\xd0\xb8\xd1\x82\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c \xd0\xbd\xd0\xb0 \xd0\xb4\xd0\xbe\xd1\x81\xd1\x82\xd0\xb0\xd1\x82\xd0\xbe\xd1\x87\xd0\xbd\xd0\xbe \xd1\x82\xd0\xb2\xd0\xb5\xd1\x80\xd0\xb4\xd1\x8b\xd0\xb9 \xd0\xb3\xd1\x80\xd1\x83\xd0\xbd\xd1\x82 \xd1\x82\xd0\xb8\xd0\xbf\xd0\xb0 \xd0\xbf\xd0\xb5\xd0\xbc\xd0\xb7\xd1\x8b.\n    \xd0\x92\xd0\xb5\xd1\x80\xd1\x82\xd0\xb8\xd0\xba. \xd1\x81\xd0\xba\xd0\xbe\xd1\x80. \xe2\x89\x88 0 \xd0\xbc/\xd1\x81\n    \xd0\xbf\xd1\x80\xd0\xb8 \xd1\x81\xd0\xbf\xd1\x83\xd1\x81\xd0\xba\xd0\xb5 \xd0\xbd\xd0\xb0 =\n    h = 1 \xd0\xbc/\xd1\x81\n    \xd0\x91\xd0\xbe\xd0\xba\xd0\xbe\xd0\xb2. \xd1\x81\xd0\xba\xd0\xbe\xd1\x80. \xd0\xb4.\xd0\xb1. \xd0\xbf\xd1\x80\xd0\xb0\xd0\xba\xd1\x82\xd0\xb8\xd1\x87\xd0\xb5\xd1\x81\xd0\xba\xd0\xb8 \xd0\xbf\xd1\x80\xd0\xb8 \xd0\xb7\xd0\xbd\xd0\xb0\xd1\x87.\n    \xe2\x89\x88 0 \xd0\xbc/\xd1\x81\n    28/X-64. \xd0\xa1.\xd0\x9a\xd0\xbe\xd1\x80\xd0\xbe\xd0\xbb\xd1\x91\xd0\xb2 [2]\n\n    \xd0\x9f\xd0\xbe\xd1\x81\xd0\xbc\xd0\xbe\xd1\x82\xd1\x80\xd0\xb8 \xd0\xb8\xd0\xb7\xd0\xbe\xd0\xb1\xd1\x80\xd0\xb0\xd0\xb6\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5: https://i.imgur.com/IGuBwsA.jpg\n\x94.